# Memento

The Memento Design Pattern, a behavioral design pattern, specializes in capturing and storing an object’s state for future retrieval without compromising its functionality.

The main parts:
- Originator: Manages the object’s state, providing methods for its storage and retrieval.
- Memento: A container capturing an object’s state, offering retrieval methods without external interference.
- Caretaker: Safeguards Mementos without altering their content, storing them for future use.


In [2]:
# memento

class Memento:
    def __init__(self, balance, operation, amount):
        self.balance = balance
        self.operation = operation
        self.amount = amount

    def get_state(self):
        return self.balance, self.operation, self.amount

In [3]:
# originator

class BankAccount:
    def __init__(self, balance):
        self.balance = balance
        self.current = 0
        self.changes = []
        self.changes.append(BankAccount.create_memento((self.balance, "open", 0)))

    @staticmethod
    def create_memento(state):
        return Memento(*state)
    
    def deposit(self, amount):
        self.balance += amount
        self.changes.append(BankAccount.create_memento((self.balance, "deposit", amount)))
        self.current += 1

    def withdraw(self, amount):
        if self.balance - amount >= 0:
            self.balance -= amount
            self.changes.append(BankAccount.create_memento((self.balance, "withdraw", amount)))
            self.current += 1
        else:
            raise ValueError("Not enough sold.")
        
    def undo(self):
        if self.current > 0:
            self.current -= 1
            self.balance, op, amount = self.changes[self.current].get_state()
            print(f"Op: {op} {amount}")

    def redo(self):
        if self.current + 1 < len(self.changes):
            self.current += 1
            self.balance, op, amount  = self.changes[self.current].get_state()
            print(f"Op: {op} {amount}")

    def __str__(self):
        return f"The sold: {self.balance}"
        

In [4]:
account = BankAccount(100)
account.deposit(200)
account.deposit(400)
account.withdraw(300)
print(account)

account.undo()
print(account)
account.undo()
print(account)
account.redo()
print(account)
account.redo()
print(account)

The sold: 400
Op: deposit 400
The sold: 700
Op: deposit 200
The sold: 300
Op: deposit 400
The sold: 700
Op: withdraw 300
The sold: 400


## with caretaker

In [19]:
class Memento:
    def __init__(self, state):
        self._state = state

    def get_state(self):
        return self._state

# originator
class TextEditor:
    def __init__(self):
        self._text = ""

    def write(self, text):
        self._text += text

    def get_text(self):
        return self._text

    def create_memento(self):
        return Memento(self._text)

    def restore(self, memento):
        self._text = memento.get_state()

# caretaker
class History:
    def __init__(self):
        self._history = []

    def push(self, memento):
        self._history.append(memento)

    def pop(self):
        if self._history:
            return self._history.pop()

In [20]:
editor = TextEditor()
history = History()

editor.write("Hello, ")
history.push(editor.create_memento())

editor.write("world!")
history.push(editor.create_memento())

print("Current Text:", editor.get_text())

editor.restore(history.pop())
print("Undo:", editor.get_text())

editor.restore(history.pop())
print("Undo:", editor.get_text())

Current Text: Hello, world!
Undo: Hello, world!
Undo: Hello, 
